In [7]:
library(data.table)
library(TwoSampleMR)
library(dplyr)
library(stringr)

In [6]:
### input all the IVs (exposure data)
workdir <- '/pathway/for/MR'
setwd(workdir)
exposure_data <- fread('combined_brain_blood_xQTL_clumped_IVs_jmm_MHCout.csv')
head(exposure_data)
dim(exposure_data)

SNP,chr.exposure,pos.exposure,effect_allele.exposure,other_allele.exposure,eaf.exposure,beta.exposure,se.exposure,pval.exposure,exposure,mr_keep.exposure,pval_origin.exposure,id.exposure,samplesize.exposure,group
<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<int>,<chr>
rs505922,9,136149229,C,T,0.313,1.298,0.014,1.0e-200,Blood_Zheng2020_pQTL_ABO,TRUE,reported,LgH5VA,3301,Blood_Zheng2020_p
rs4344,17,61566724,A,G,0.499,-0.583,0.425,8.5e-136,Blood_Zheng2020_pQTL_ACE,TRUE,reported,3eU1Rv,3200,Blood_Zheng2020_p
rs4727469,7,100509163,C,T,0.524,-0.297,0.463,7.5e-37,Blood_Zheng2020_pQTL_ACHE,TRUE,reported,pxl3CF,3200,Blood_Zheng2020_p
rs11553746,2,272203,T,C,0.325,1.237,0.026,1.0e-200,Blood_Zheng2020_pQTL_ACP1,TRUE,reported,SM7429,996,Blood_Zheng2020_p
rs79061565,19,11698659,G,C,0.123,-0.485,0.036,1.6e-40,Blood_Zheng2020_pQTL_ACP5,TRUE,reported,CgRhmO,3301,Blood_Zheng2020_p
rs2153463,1,147124310,G,T,0.739,0.944,0.695,1.0e-200,Blood_Zheng2020_pQTL_ACP6,TRUE,reported,KLTPoh,3200,Blood_Zheng2020_p


[1] 190359     15

In [ ]:
### input outcome data
cjd_gwas <- fread('GCST90001389_buildGRCh37_rsID.tsv.gz')
merC <- merge(exposure_data,cjd_gwas,by.x = 'SNP', by.y = 'name')    
outcome_data <-format_data(
  merC,
  type = 'outcome',
  snps = exposure_data$SNP,
  snp_col = 'SNP',
  beta_col = 'beta',
  se_col = 'standard_error',
  effect_allele_col = 'effect_allele',
  other_allele_col = 'other_allele',
  pval_col = 'p_value')  


In [ ]:
### harmonise
H_data <- harmonise_data(
  exposure_dat = exposure_data,
  outcome_dat = outcome_data,
  action = 2
)
write.csv(H_data,file='Harmo_CJD_all_xQTL.csv',row.names = FALSE)


In [ ]:
# steiger_filtering
H_data <- fread('Harmo_CJD_all_mQTL.csv')
H_data <- steiger_filtering(H_data)
H_data <- subset(H_data,H_data$steiger_dir == 'TRUE')



In [ ]:
# MR analysis
mr_results<-generate_odds_ratios(mr(H_data,
    method_list=c('mr_egger_regression_bootstrap',"mr_wald_ratio", "mr_ivw", "mr_weighted_median", "mr_weighted_mode",'mr_two_sample_ml')))
mr_results$outcome <- 'sCJD'
mr_results <- mr_results[,c(-1,-2)]
write.csv(mr_results,file='MRresults_CJD_all_mQTL.csv',row.names = FALSE)



In [ ]:
## pleiotropy_test
mr_pleio <- mr_pleiotropy_test(H_data)
    if (length(mr_pleio) > 0) {
      mr_pleio$outcome <- 'sCJD'
      mr_pleio <- mr_pleio[, c(-1, -2)]
    }
write.csv(mr_pleio,file'MRpleio_CJD_all_mQTL.csv',row.names = FALSE)



In [ ]:
## heterogeneity
mr_het <- mr_heterogeneity(H_data)
    if (length(mr_het) > 0) {
      mr_het$outcome <- 'sCJD'
      mr_het <- mr_het[, c(-1, -2)]
    }
write.csv(mr_het,file='MRhet_CJD_all_mQTL.csv',row.names = FALSE)

In [ ]:
### FDR adjustment
outresult <- fread('MRresults_CJD_all_mQTL.csv')

split_cols <- strsplit(outresult$exposure, "_")
outresult$group <- sapply(split_cols, function(x) paste(x[-length(x)], collapse = "_"))
outresult <- outresult %>% 
  mutate(group = gsub("Blood_GTEx_eQTL_Whole_Blood.*", "Blood_GTEx_eQTL_Whole_Blood", group)) %>%
  mutate(group = gsub("Brain_GTEx_eQTL_Cerebellum.*", "Brain_GTEx_eQTL_Cerebellum", group)) %>%
  mutate(group = gsub("Brain_GTEx_eQTL_Cortex.*", "Brain_GTEx_eQTL_Cortex", group)) %>%
  mutate(group = gsub("Brain_GTEx_eQTL_Cerebellar_Hemisphere.*", "Brain_GTEx_eQTL_Cerebellar_Hemisphere", group)) %>%
  mutate(group = gsub("Brain_GTEx_eQTL_Hippocampus.*", "Brain_GTEx_eQTL_Hippocampus", group)) %>%
  mutate(group = gsub("Brain_GTEx_eQTL_Nucleus_accumbens_basal_ganglia.*", "Brain_GTEx_eQTL_Nucleus_accumbens_basal_ganglia", group)) %>%
  mutate(group = gsub("Brain_GTEx_eQTL_Putamen_basal_ganglia.*", "Brain_GTEx_eQTL_Putamen_basal_ganglia", group)) %>%
  mutate(group = gsub("Brain_PsychENCODE_eqtl.*", "Brain_PsychENCODE_eqtl", group)) %>%
  mutate(group = gsub("Brain_BrainMeta_eqtl.*",  "Brain_BrainMeta_eqtl", group))
      
outresult_only <- subset(outresult,method == 'Wald ratio'| method == 'Inverse variance weighted')

outresult_only <- outresult_only %>% 
  group_by(group) %>% 
  mutate(FDR_Pvalue = p.adjust(pval, method = "fdr"))
fdrout <- subset(outresult_only,FDR_Pvalue < 0.05)
newdat <- outresult_only[,c(2,14)]
merged_df <- merge(outresult, newdat, by = "exposure", all.x = TRUE, allow.cartesian = TRUE, nomatch = NA)

write.csv(merged_df,file='TSMR_risk_CJD_FDR.csv',row.names=FALSE)
write.csv(fdrout,file='TSMR_risk_CJD_FDR005.csv',row.names=FALSE)
